In [2]:
import time
import pandas as pd
import numpy as np
import talib
import json
import datetime
import telegram
from twilio.rest import Client
import openpyxl
from kiteconnect import KiteConnect
from apscheduler.schedulers.blocking import BlockingScheduler
import yfinance as yf
import requests

# Initialize Zerodha API
api_key = "your_api_key"
access_token = "your_access_token"
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token)

print("\n🚀 Options Trading Bot Started... Press Ctrl+C to stop.\n")

# Function to fetch Option Chain data
def get_option_chain(symbol):
    data = kite.ltp([f"NSE:{symbol}"])
    return data[f"NSE:{symbol}"]['last_price']

# Fetch top gainers/losers for automatic stock selection
def get_top_stocks():
    url = "https://www.nseindia.com/api/live-analysis-variations"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    data = response.json()
    return [stock["symbol"] for stock in data["data"][:10]]  # Top 10 stocks

# Technical Indicators & Patterns
def check_candle_patterns(df):
    patterns = {
        "Bullish Engulfing": talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close']) > 0,
        "Bearish Engulfing": talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close']) < 0,
        "Morning Star": talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close']) > 0,
        "Evening Star": talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close']) > 0,
    }
    return [key for key, value in patterns.items() if value.any()]

# Trading Logic
def trade_options(symbol):
    df = yf.download(symbol, period='7d', interval='15m')
    patterns = check_candle_patterns(df)
    if patterns:
        option_price = get_option_chain(symbol)
        entry_price = option_price
        stop_loss = entry_price * 0.98  # 2% SL
        take_profit = entry_price * 1.05  # 5% TP
        trade_details = {
            "Stock": symbol,
            "Entry Price": entry_price,
            "Stop Loss": stop_loss,
            "Take Profit": take_profit,
            "Patterns Matched": patterns,
        }
        send_trade_alert(trade_details)
        save_trade_log(trade_details)
        print_trade_output(trade_details)

# Sending Alerts
def send_trade_alert(trade_details):
    message = f"📈 Trade Alert:\n{json.dumps(trade_details, indent=2)}"
    bot = telegram.Bot(token="your_telegram_token")
    bot.send_message(chat_id="your_chat_id", text=message)
    client = Client("twilio_sid", "twilio_auth_token")
    client.messages.create(to="your_phone_number", from_="twilio_phone", body=message)

# Save Trade Data
def save_trade_log(trade_details):
    df = pd.DataFrame([trade_details])
    df.to_csv("trade_log.csv", mode='a', header=False, index=False)

# Print Output
def print_trade_output(trade_details):
    print("-------------------------------------------")
    print("📊 Options Trade Executed:")
    for key, value in trade_details.items():
        print(f"{key}: {value}")
    print("-------------------------------------------")

# Scheduler to Run the Bot Every 5 Min
scheduler = BlockingScheduler()
scheduler.add_job(lambda: [trade_options(stock) for stock in get_top_stocks()], 'interval', minutes=5)
scheduler.start()



🚀 Options Trading Bot Started... Press Ctrl+C to stop.



KeyboardInterrupt: 